# Install Requirements

In [1]:
!git clone https://github.com/Building-ML-Pipelines/building-machine-learning-pipelines

Cloning into 'building-machine-learning-pipelines'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 740 (delta 31), reused 36 (delta 13), pack-reused 663
Receiving objects: 100% (740/740), 29.41 MiB | 10.22 MiB/s, done.
Resolving deltas: 100% (388/388), done.


In [2]:
%cd drive/My\ Drive/Building\ ML\ Pipelines/
!pip install -r requirements.txt
%cd ..
%cd ..

/content/drive/My Drive/Building ML Pipelines
     |████████████████████████████████| 1.8MB 2.7MB/s 
     |████████████████████████████████| 153kB 14.7MB/s 
     |████████████████████████████████| 2.3MB 13.8MB/s 
     |████████████████████████████████| 8.6MB 34.1MB/s 
     |████████████████████████████████| 81kB 7.4MB/s 
     |████████████████████████████████| 63.8MB 61kB/s 
     |████████████████████████████████| 1.6MB 44.5MB/s 
     |████████████████████████████████| 1.8MB 43.2MB/s 
     |████████████████████████████████| 1.5MB 44.8MB/s 
     |████████████████████████████████| 378kB 36.7MB/s 
     |████████████████████████████████| 1.3MB 38.3MB/s 
     |████████████████████████████████| 153kB 38.2MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 2.8MB 33.7MB/s 
     |████████████████████████████████| 92kB 8.1MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 1.4MB 25.5MB/s 
     |██████

/content/drive/My Drive
/content/drive


# Chapter 5: Data Preprocessing

In [1]:
import tensorflow as tf

base_dir = "drive/My Drive/Building ML Pipelines/"
chap_dir = base_dir + "Chapter 5/"
data_dir = base_dir + "Data/"
out_dir = chap_dir + "Outputs/"
csv_data_dir = base_dir + "CSV Data/"
csv_dir = csv_data_dir + "consumer_complaints_with_narrative.csv"

## Why Data Preprocessing?

 - Convert Data into consistent numerical representations of the features
 - Preprocessing the Data in the Context of the Entire Dataset
 - Sacling the Preprocessing steps
 - Avoiding a potential Training-Serving Skew (Use TFT to reduce amount of coordination needed)
 - Deployment of Preprocessing Steps and the ML Model as one Artifact (to avoid missalignments)
 - Checking Preprocessing Results in the Pipeline


#### **The Difference Between tf.data and tf.transform**

tf.data is a TensorFlow API for building efficient input pipelines for model training with TensorFlow while utilizing hardware resources such as CPU and RAM for data ingestion and preprocessing optimally during training.

tf.transform is used to express preprocessing that should happen both in training and inference time. The library makes it possible to perform a full-pass anaylsis of the input data which is executed ahead of training.

## Data Preprocessing with TFT

TFT processes the data that we ingested into our pipeline with the earlier generated dataset schema, and it outputs two artifacts:
 - Preprocessed training and ealuation datasets in the TFRecord Format
 - Exported preprocessing graph (with assets), which will be used when we will export our machine learning model

#### **TFT Functions**

TFT functions perform elaborate processing steps behind the scence like *tft.compute_and_apply_vocabulary*.

**Applied transformations are defined in a function called *preprocessing_fn()***

### Installation

In [2]:
# If tfx was not installed as above run this line
# !pip install tensorflow-transform

### Preprocessing Strategies

In [3]:
import tensorflow_transform as tft

In [4]:
# Example of preprocessing function
def preprocessing_fn(inputs): # inputs is in python dictionary format
    x = inputs["x"]
    x_normalized = tft.scale_to_0_1(x)
    return {
        "x_xf": x_normalized
    }

**Note:** TFT will build a graph out of the preprocessing_fn() function and it will run in its own session. It is expected that the function returns a dictionary with the transformed features as values of the Python dictionary.

### Best Practices

 - Feature names matter
 - Consider the data types
 - Preprocessing happens in batches
 - Remember, no eager execution

### TFT Functions

Some example functions which can be useful:

 - tft.scale_to_z_score() , to normalize a feature
 - tft.bucketsize() , to bucketsize a feature into bins
 - tft.pca() , computes PCA for a given feature
 - tft.compute_and_apply_vocabulary() , computes all unique values of a feature column and the maps the most frequent values to an index (used to convert the feature to a numerical representation)
 - tft.apply_save_model() , Apply entire TensorFlow models on a feature

#### Text data for natural language problems

See, Chapter 5.2 in the Book Building Machine Learning Pipelines or look at <a href="https://github.com/tensorflow/text">TensorFlow Text - Text processing in TensorFlow</a> for more text related functions.

#### Image data for computer vision problems

TensorFlow's provided image preprocessing operations:
 - <a href="https://www.tensorflow.org/api_docs/python/tf/image">tf.images</a> API
 - <a href="https://www.tensorflow.org/api_docs/python/tf/io">tf.io</a> API




In [5]:
# Example preprocessing_fn function for images
def process_image(raw_image):
    raw_image = tf.reshape(raw_iamge, [-1])
    img_rgb = tf.io.decode_jpg(raw_image, channels=3)
    img_gray = tf.image.rgb_to_grayscale(img_rgb)
    img = tf.image.convert_image_dtype(img_gray, tf.float32)
    resized_img = tf.image.resize_with_pad(img, target_height=300, target_width=300)
    img_grayscale = tf.image.rgb_to_grayscale(resized_img)
    
    return tf.reshape(img_grayscale, [-1, 300, 300, 1])

### Standalone Execution of TFT

There are two options to execute preprocessing functions:
 - Standalone Setup
 - Part of a ML pipeline in the form of a TFX component

Both forms can be executed on a local Apache Beam Setup or on Google Cloud's Dataflow service.



In [6]:
# Apply normalization preprocesing function from above on a small example dataset
raw_data = [
            {"x": 1.20},
            {"x": 2.99},
            {"x": 100.0}
]

In [7]:
import tensorflow as tf
from tensorflow_transform.tf_metadata import dataset_metadata, schema_utils

In [8]:
# First, define a data schema
# Our dataset only contains one feature 'x', which we define with the tf.float32 data type
raw_data_metadata = dataset_metadata.DatasetMetadata(
    schema_utils.schema_from_feature_spec({
        "x": tf.io.FixedLenFeature([], tf.float32),
    })
)

print(raw_data_metadata)

{'_schema': feature {
  name: "x"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
}


In [9]:
import tempfile
import apache_beam as beam
import tensorflow_transform.beam.impl as tft_beam

In [10]:
with beam.Pipeline() as p:
    with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
        tf_record_file = ""
        # raw_data = (
        #     p | beam.io.ReadFromTFRecord(tfrecord_file)
        # )

        raw_dataset = (raw_data, raw_data_metadata)
        
        transformed_dataset, transform_fn = (
            raw_dataset | tft_beam.AnalyzeAndTransformDataset(preprocessing_fn)
        )

        transformed_data, transformed_metadata = transformed_dataset
        print(transformed_data)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


INFO:tensorflow:Assets added to graph.


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


[{'x_xf': 0.0}, {'x_xf': 0.018117407}, {'x_xf': 1.0}]


### Intergrate TFT into Your Machine Learning Pipeline

In [11]:
import tensorflow as tf
import tensorflow_transform as tft

In [12]:
# Define the features
# For simpler processing later on, we group the input feature names in dictionaries
# representing each transform output data type:
# one-hot encoded features, bucketized features, and raw string representations

LABEL_KEY = "consumer_disputed"

# Feature name, feature dimensionality
ONE_HOT_FEATURES = {
    "prodcut": 11,
    "sub_product": 45,
    "company_response": 5,
    "state": 60,
    "issue": 90
}

# Feature name, bucket count
BUCKET_FEATURES = {
    "zip_code": 10
}

# Feature name, value is unused
TEXT_FEATURES = {
    "consumer_complaint_narrative": None
}

In [13]:
# Define helper functions to transform data efficiently
# It is a good practice to rename the features by appending a suffix to the feature name (e.g. _xf)
# The suffix will help distinguish whether errors are originatig from input or output features
def transformed_name(key):
    return key + "_xf"


def fill_in_missing(x):
    default_value = '' if x.dtype == tf.string or to_string else 0
    if type(x) == tf.SparseTensor:
        x = tf.sparse.to_dense(
            tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]), default_value
        )

    return tf.squeeze(x, axis=1)


def convert_num_to_one_hot(label_tensor, num_labels=2):
    one_hot_tensor = tf.one_hot(label_tensor, num_labels)
    
    return tf.reshape(one_hot_tensor, [-1, num_labels])


def convert_zip_code(zip_code):
    if zip_code == "":
        zip_code = "00000"

    zip_code = tf.strings.regex_replace(zip_code, r"X{0,5}", "0")
    zip_code = tf.strings.to_number(zip_code, out_type=tf.float32)

    return zip_code

In [14]:
# With all helper functions in place we can now loop over each feature column and 
# transform it depending on the type
def preprocessing_fn(inputs):
    outputs = {}
    for key in ONE_HOT_FEATURES.keys():
        dim = ONE_HOT_FEATURES[key]
        index = tft.compute_and_apply_vocabulary(
            fill_in_missing(inputs[key]),
            top_k=dim + 1
        )

        outputs[transformed_name(key)] = convert_num_to_one_hot(index, num_labels=dim + 1)

    for key, bucket_count in BUCKET_FEATURES.items():
        temp_feature = tft.bucketize(
            convert_zip_code(fill_in_missing(inputs[key])),
            bucket_count,
            always_return_num_quantiles=False
        )
        outputs[transformed_name(key)] = convert_num_to_one_hot(
            temp_feature,
            num_labels=bucket_count + 1
        )

    for key in TEXT_FEATURES.keys():
        outputs[transformed_name(key)] = fill_in_missing(inputs[key])

    outputs[transformed_name(LABEL_KEY)] = fill_in_missing(inputs[LABEL_KEY])

    return outputs

**Why We Didn't Embed the Text Features to Vectors**

This makes the embeddings trainable and refine the vector representations during our training phase.
Therefore it can not be hard-coded into the preprocessing step and represented as a fixed graph during the training phase.

If the Transform compoment from TFX is used in the pipeline, it expects the transformation code to be provided in a separate Python file.

In [24]:
from tfx.components import Transform, CsvExampleGen, SchemaGen, StatisticsGen
from tfx.utils.dsl_utils import external_input
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

In [29]:
examples = external_input(csv_data_dir)
example_gen = CsvExampleGen(input=examples)
context.run(example_gen)


statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)


schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)
context.run(schema_gen)


trainer_file = "building-machine-learning-pipelines/components/module.py"


transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=trainer_file
)
context.run(transform)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`


INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: drive/My Drive/Building ML Pipelines/Chapter 5/Outputs/Transform/transform_graph/12/.temp_path/tftransform_tmp/d12af44c04404a44b54df3c39dfc43fd/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: drive/My Drive/Building ML Pipelines/Chapter 5/Outputs/Transform/transform_graph/12/.temp_path/tftransform_tmp/4de630a7394948be9035d40749811907/saved_model.pb


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:Assets written to: drive/My Drive/Building ML Pipelines/Chapter 5/Outputs/Transform/transform_graph/12/.temp_path/tftransform_tmp/f1d421985e624d3e81629cdc7d6057fa/assets
INFO:tensorflow:SavedModel written to: drive/My Drive/Building ML Pipelines/Chapter 5/Outputs/Transform/transform_graph/12/.temp_path/tftransform_tmp/f1d421985e624d3e81629cdc7d6057fa/saved_model.pb
value: "\n\013\n\tConst_1:0\022-vocab_compute_and_apply_vocabulary_vocabulary"

value: "\n\013\n\tConst_3:0\022/vocab_compute_and_apply_vocabulary_1_vocabulary"

value: "\n\013\n\tConst_5:0\022/vocab_compute_and_apply_vocabulary_2_vocabulary"

value: "\n\013\n\tConst_7:0\022/vocab_compute_and_apply_vocabulary_3_vocabulary"

value: "\n\013\n\tConst_9:0\022/vocab_compute_and_apply_vocabulary_4_voca

ExecutionResult(
    component_id: Transform
    execution_id: 12
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 9
        type_id: 11
        uri: "drive/My Drive/Building ML Pipelines/Chapter 5/Outputs/Transform/transform_graph/12"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 11
        name: "TransformGraph"
        )]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 10
        type_id: 5
        uri: "drive/My Drive/Building ML Pipelines/Chapter 5/Outputs/Transform/transformed_examples/12"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 11
        type_id: 12
        uri: "drive/My Drive/Building ML Pipelines/Chapter 5/Outputs/Transform/updated_analyzer_cache/12"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 12
        name: "TransformCache"
        )]
        ))

## References

 - <a href="https://github.com/tensorflow/transform">TensorFlow Transform Github</a>
 - <a href="https://www.tensorflow.org/tfx/transform/get_started">TensorFlow Transform</a>
 - <a href="https://github.com/tensorflow/text">TensorFlow Text</a>
 - <a href="https://www.tensorflow.org/api_docs/python/tf/image">tf.images</a> API
 - <a href="https://www.tensorflow.org/api_docs/python/tf/io">tf.io</a> API